In [1]:
import requests 
import csv
import json
import pandas as pd
from datetime import datetime, timezone
import os
from pathlib import Path



In [2]:
# declaring the output path
output_path = "D:/products/ALERT_DTC_DOWNLOAD"

In [3]:
# internal functions being used
def trip_data_download(vehicle_id, start_ts, end_ts):
    trip_url = 'http://internal-apis.intangles.com/trips/' + str(vehicle_id) + '/gettripsintime/' + str(start_ts) + '/' + str(end_ts) + '?proj=_id,start_time,end_time'
    # getting the trip-data
    trip_response = requests.get(trip_url, json=trip_url)
    if trip_response.status_code == 200:
        # Parse the JSON response
        trip_json_data = trip_response.json()
        return trip_json_data['result']
    return []

def fetch_dtc_data(start_ts, end_ts, vehicle_id):
    
    df_all_dtcs = pd.DataFrame()
    start_batch_time = start_ts 
    end_batch_time = start_batch_time + 24*1*60*60*1000 

    
    while end_batch_time <= end_ts:
        print("starting timestamp ", start_batch_time) 
        print("ending timestamp ", end_batch_time)
        headers = {
                'Content-Type': 'application/json',
            }
        vehicle_dtc_data = {"report": "default",
            "filter":[
                {
                    "fault_log.timestamp":{
                        "gt": start_batch_time,
                        "lt": end_batch_time
                    }
                },
                {
                    "fault_log.vehicle_id" : vehicle_id
                }
            ],
            "select":{
                "fault_log.status":{
                    "value":True,
                    "as":"status"
                },
                "fault_log.code":{
                    "value":True,
                    "as":"code"
                },
                "fault_code.severity":{
                    "value":True,
                    "as":"severity"
                },
                "vehicle.id":{
                    "value":True,
                    "as":"vehicle_id"
                },
                "fault_log.vehicle_id":{
                    "value":True,
                    "as":"vehicle_id1"
                },
                "vehicle.account_id": {
                    "value": True,
                    "as": "account_id"
                },
                "vehicle.tag":{
                    "value":True,
                    "as":"vehicle_plate"
                },
                "fault_log.timestamp":{
                    "value":True,
                    "as":"time"
                },
                "spec.manufacturer":{
                    "value":True,
                    "as":"manufacturer"
                },
                "spec.max_load_capacity":{
                    "value":True,
                    "as": "max_load_capacity"
                },
                "fault_code.description":{
                    "value":True,
                    "as":"description"
                }
            }
            }
        dtc_url = 'http://internal-apis.intangles.com/dashboard_apis/fetch'
        dtc_response = requests.post(dtc_url, json=vehicle_dtc_data, headers=headers)
        print(dtc_response)
        dtc_csv_filename = 'dtc.csv'
        if dtc_response.status_code == 200:
            # Parse the JSON response
            dtc_json_data = dtc_response.json()
            dtc_headers = dtc_json_data['result']['fields']
            with open(dtc_csv_filename, 'w', newline='') as csv_file:
                csv_writer = csv.DictWriter(csv_file, fieldnames=dtc_headers)
                
                # Write headers to the CSV file
                csv_writer.writeheader()
                
                # Write each JSON item as a row in the CSV file
                csv_writer.writerows(dtc_json_data['result']['output'])
        else:
            print(f"Failed to fetch the dtc data. Status code: {dtc_response.status_code}")
        df_dtc = pd.read_csv(dtc_csv_filename, encoding='unicode_escape') 

        start_batch_time += 24*1*60*60*1000 
        end_batch_time = start_batch_time + 24*1*60*60*1000

        df_all_dtcs= pd.concat([df_all_dtcs, df_dtc], ignore_index=True)

    return df_all_dtcs



def fetch_alert_data(start_ts, end_ts, vehicle_id):
    
   

        headers = {
            'Content-Type': 'application/json',
        }


        vehicle_alert_data = {
            "report": "dafault",
            "filter": [
                {
                    "alert_algo_output.timestamp": {
                        "gt": start_ts,
                        "lt": end_ts
                    }
                },
                {
                    "vehicle.id" : vehicle_id
                }
               
    
            ],
            "select": {
                "alert_algo_output.account_id": {
                    "value": True,
                    "as": "account_id"
                },
                "alert_algo_output.vehicle_id": {
                    "value": True,
                    "as": "vehicle_id"
                },
                "alert_algo_output.severity": {
                    "value": True,
                    "as": "severity"
                },
                "alert_algo_output.alerts": {
                    "value": True,
                    "as": "alert_name"
                },
                "alert_algo_output.timestamp": {
                    "value": True,
                    "as": "time"
                },
                "vehicle.spec_id":{
                    "value":True,
                    "as":"spec_id"
                },
                "spec.model":{
                    "value":True,
                    "as":"model"
                },
                "spec.manufacturer":{
                    "value":True,
                    "as":"manufacturer"
                },
                "spec.max_load_capacity":{
                    "value":True,
                    "as":"max_load_capacity"
                },
                
                "vehicle.id":{
                    "value":True,
                    "as":"vehicle_id"
                }
            }
        }

        

        alert_url = 'http://internal-apis.intangles.com/dashboard_apis/fetch'

        # getting the alert-data
        alert_response = requests.post(alert_url, json=vehicle_alert_data, headers=headers)
        alert_csv_filename = 'alert.csv'
        if alert_response.status_code == 200:
            # Parse the JSON response
            json_data = alert_response.json()
            alert_headers = json_data['result']['fields']
            with open(alert_csv_filename, 'w', newline='') as csv_file:
                csv_writer = csv.DictWriter(csv_file, fieldnames = alert_headers)
                
                # Write headers to the CSV file
                csv_writer.writeheader()
                
                # Write each JSON item as a row in the CSV file
                csv_writer.writerows(json_data['result']['output'])
        else:
            print(f"Failed to fetch the alert data. Status code: {alert_response.status_code}")
        df_alert = pd.read_csv(alert_csv_filename, encoding='unicode_escape') 
        


        
        return df_alert

def miliseconds_to_utc(time_ms):
    time_seconds = time_ms / 1000.0
    # Create a UTC datetime object
    trip_time_utc = datetime.utcfromtimestamp(time_seconds).replace(tzinfo=timezone.utc)
    formatted_trip_time_utc = trip_time_utc.strftime('%Y-%m-%dT%H:%M:%S.%fZ')

    return formatted_trip_time_utc

def utc_to_miliseconds(utc_date_string):
    # Parse the UTC date string into a datetime object
    utc_datetime = datetime.strptime(utc_date_string, '%Y-%m-%dT%H:%M:%S.%fZ')
    # Convert seconds to milliseconds
    milliseconds = int(utc_datetime.timestamp() * 1000)

    return milliseconds





In [7]:
# enter the start time and end time between which you want the trips for vehicles
vehicle_id = 1294352743401521152
start_ts = 1718821800000
end_ts = 1719685740000
df_dtc = fetch_dtc_data(start_ts, end_ts, vehicle_id)


starting timestamp  1718821800000
ending timestamp  1718908200000
<Response [200]>
starting timestamp  1718908200000
ending timestamp  1718994600000
<Response [200]>
starting timestamp  1718994600000
ending timestamp  1719081000000
<Response [200]>
starting timestamp  1719081000000
ending timestamp  1719167400000
<Response [200]>
starting timestamp  1719167400000
ending timestamp  1719253800000
<Response [200]>
starting timestamp  1719253800000
ending timestamp  1719340200000
<Response [200]>
starting timestamp  1719340200000
ending timestamp  1719426600000
<Response [200]>
starting timestamp  1719426600000
ending timestamp  1719513000000
<Response [200]>
starting timestamp  1719513000000
ending timestamp  1719599400000
<Response [200]>


In [8]:
df_dtc

,status,code,severity,vehicle_id,vehicle_id1,account_id,vehicle_plate,time,manufacturer,max_load_capacity,description


In [6]:
# downloading the alerts and dtcs for all the vehicle_id between the start and end time-stamp
df_alert = fetch_alert_data(start_ts, end_ts, vehicle_id)
df_alert['time_miliseconds'] = df_alert['time'].apply(utc_to_miliseconds)

In [39]:
df_alert

,account_id,vehicle_id,severity,alert_name,time,spec_id,model,manufacturer,max_load_capacity,vehicle_id.1,time_miliseconds


In [21]:
df_dtc = fetch_dtc_data(start_ts, end_ts)
df_dtc = df_dtc.drop_duplicates(keep='first')
df_dtc['time_miliseconds'] = df_dtc['time'].apply(utc_to_miliseconds)

starting timestamp  1693333800000
ending timestamp  1693420200000
<Response [200]>
starting timestamp  1693506600000
ending timestamp  1693593000000
<Response [200]>
starting timestamp  1693679400000
ending timestamp  1693765800000
<Response [200]>
starting timestamp  1693852200000
ending timestamp  1693938600000
<Response [200]>
starting timestamp  1694025000000
ending timestamp  1694111400000
<Response [200]>
starting timestamp  1694197800000
ending timestamp  1694284200000
<Response [200]>
starting timestamp  1694370600000
ending timestamp  1694457000000


KeyboardInterrupt: 

In [7]:
df_dtc = df_dtc[(df_dtc.manufacturer == manufacturer) & (df_dtc.max_load_capacity == max_load_capacity)]
df_alert = df_alert[(df_alert.manufacturer == manufacturer) & (df_alert.max_load_capacity == max_load_capacity)]
df_alert.head()

,account_id,vehicle_id,severity,alert_name,time,spec_id,model,manufacturer,max_load_capacity,time_miliseconds
21,999705020632924160,951065816231575552,Major,Engine operating temperature high,2023-08-31T05:48:07.000Z,1027240614976028672,furio 16T cargo BS6,mahindra,16000.0,1693441087000
31,993207809577320448,927139700064387072,Minor,Engine operating temperature high,2023-09-28T01:27:08.000Z,848542417341644800,furio 16T cargo BS6,mahindra,16000.0,1695844628000
49,1048293817746194432,1013067916804882432,Minor,Engine operating temperature high,2023-09-28T00:29:29.000Z,1027240614976028672,furio 16T cargo BS6,mahindra,16000.0,1695841169000
63,951132064558612480,918062870334275584,Minor,Engine operating temperature high,2023-08-31T11:15:49.000Z,1084570912415547392,furio 16T cargo BS6,mahindra,16000.0,1693460749000
151,914116281580388352,887606546836488192,Minor,Engine operating temperature high,2023-08-30T20:15:38.000Z,787995314035359744,furio 16T cargo BS6,mahindra,16000.0,1693406738000


In [8]:
df_dtc.head()

,status,code,severity,vehicle_id,vehicle_id1,account_id,vehicle_plate,time,manufacturer,max_load_capacity,description,time_miliseconds
9,active,810-14,1.0,1000041832370929664,1000041832370929664,1028649932849938432,UP 75 BT 6232,2023-08-30T09:56:51.000Z,mahindra,16000.0,Speed Signal Input,1693369611000
10,active,P2454,1.0,1000041832370929664,1000041832370929664,1028649932849938432,UP 75 BT 6232,2023-08-30T01:50:49.000Z,mahindra,16000.0,Particulate Filter Pressure sensor output belo...,1693340449000
11,active,P2454,2.0,1000041832370929664,1000041832370929664,1028649932849938432,UP 75 BT 6232,2023-08-30T01:50:49.000Z,mahindra,16000.0,Particulate Filter Pressure sensor output belo...,1693340449000
12,active,P2454,1.0,1000041832370929664,1000041832370929664,1028649932849938432,UP 75 BT 6232,2023-08-29T18:48:16.000Z,mahindra,16000.0,Particulate Filter Pressure sensor output belo...,1693315096000
13,active,P2454,2.0,1000041832370929664,1000041832370929664,1028649932849938432,UP 75 BT 6232,2023-08-29T18:48:16.000Z,mahindra,16000.0,Particulate Filter Pressure sensor output belo...,1693315096000


In [9]:
# getting the alerts and trips mapped
unique_vehicles = df_alert.vehicle_id.unique().tolist()
print(len(unique_vehicles))
df_all_trip_alerts = pd.DataFrame()
count = 0
for vehicle_idx in range(len(unique_vehicles)):
    # downloading all the trips of the vehicle_id between the start and end time-stamp
    vehicle = unique_vehicles[vehicle_idx]
    trip_json_data = trip_data_download(vehicle, start_ts, end_ts)
    # Convert JSON data to a DataFrame
    df_vehicle_trips = pd.DataFrame(trip_json_data)
    df_vehicle_alerts = df_alert[(df_alert.vehicle_id == vehicle)]

    if (df_vehicle_alerts.shape[0] >0) and (df_vehicle_trips.shape[0] > 0):
        for alert_idx in range(0, df_vehicle_alerts.shape[0]):
            time_alert = df_vehicle_alerts.iloc[alert_idx]['time_miliseconds']
            df_alert_trip = df_vehicle_trips.loc[(df_vehicle_trips['start_time'] <= time_alert) &
                                                   (df_vehicle_trips['end_time'] >= time_alert)]
           
           
            df_vehicle_alert = df_vehicle_alerts.iloc[alert_idx].to_frame().T
        
           
         
            if df_alert_trip.shape[0]>0:
                #df_all_trip_alerts= pd.concat([df_all_trip_alerts, df_alert_concat], ignore_index=True)
                df_temp_alert = pd.DataFrame({"Trip_ID" : [df_alert_trip.iloc[0]['_id']],
                                                "Start_Time" : [df_alert_trip.iloc[0]['start_time']],
                                                "End_Time" : [df_alert_trip.iloc[0]['end_time']],
                                                "Predictive_alerts" : [df_vehicle_alert.iloc[0]['alert_name']],
                                                "severity": [df_vehicle_alert.iloc[0]['severity']],
                                                "time_of alert": [df_vehicle_alert.iloc[0]['time']],
                                                "model": [df_vehicle_alert.iloc[0]['model']],
                                                "manufacturer": [df_vehicle_alert.iloc[0]['manufacturer']],
                                                "max_load_capacity": [df_vehicle_alert.iloc[0]['max_load_capacity']]
                                                 
                                                
                                              })

                
                df_all_trip_alerts= pd.concat([df_all_trip_alerts, df_temp_alert], ignore_index=True)
         
          
                
             
           

          
                            
                                                          
        
        
        

        

236


In [11]:
# getting the alerts and trips mapped
df_all_trip_dtcs = pd.DataFrame()
# getting the unique vehicles
unique_vehicles = df_dtc.vehicle_id.unique().tolist()
print(len(unique_vehicles))
count = 0
for vehicle_idx in range(len(unique_vehicles)):
    # downloading all the trips of the vehicle_id between the start and end time-stamp
    vehicle = unique_vehicles[vehicle_idx]
    print(vehicle)
    trip_json_data = trip_data_download(vehicle, start_ts, end_ts)
    if len(trip_json_data)>0:
        # Convert JSON data to a DataFrame
        df_vehicle_trips = pd.DataFrame(trip_json_data)
        df_vehicle_dtcs = df_dtc[(df_dtc.vehicle_id == vehicle)]
        if (df_vehicle_dtcs.shape[0] >0) and (df_vehicle_trips.shape[0] > 0):
            for alert_idx in range(0, df_vehicle_dtcs.shape[0]):
                time_dtc = df_vehicle_dtcs.iloc[alert_idx]['time_miliseconds']
                df_dtc_trip = df_vehicle_trips.loc[(df_vehicle_trips['start_time'] <= time_dtc) &
                                                    (df_vehicle_trips['end_time'] >= time_dtc)]
            
                df_vehicle_dtc = df_vehicle_dtcs.iloc[alert_idx].to_frame().T
            
            
            
                if df_dtc_trip.shape[0]>0:
        
                    df_temp_dtc = pd.DataFrame({"Trip_ID" : [df_dtc_trip.iloc[0]['_id']] ,
                                                    "Start_Time" : [df_dtc_trip.iloc[0]['start_time']],
                                                    "End_Time" : [df_dtc_trip.iloc[0]['end_time']],
                                                    "DTC" : [df_vehicle_dtc.iloc[0]['code']],
                                                    "status": [df_vehicle_dtc.iloc[0]['status']],
                                                    "time_of_dtc": [df_vehicle_dtc.iloc[0]['time']],
                                                    "description" : [df_vehicle_dtc.iloc[0]['description']]
                                                    
                                                    
                                                })
                    
                    df_all_trip_dtcs= pd.concat([df_all_trip_dtcs, df_temp_dtc], ignore_index=True)
    print("this is the vehicle index", vehicle_idx)                
                  
                
            

          
                            
                                                          
        
        
        

        

533
1000041832370929664
this is the vehicle index 0
1001786996651196416
this is the vehicle index 1
1002155382019194880
this is the vehicle index 2
1004323703212736512
this is the vehicle index 3
1003204815926329344
this is the vehicle index 4
1008371704788942848
this is the vehicle index 5
1009400932716249088
this is the vehicle index 6
1012625060713725952
this is the vehicle index 7
1012994619849310208
this is the vehicle index 8
1012995656710619136
this is the vehicle index 9
1013396739543007232
this is the vehicle index 10
1015566071555424256
this is the vehicle index 11
1016287725000589312
this is the vehicle index 12
1016336951592091648
this is the vehicle index 13
1016337068688670720
this is the vehicle index 14
1020634464985808896
this is the vehicle index 15
1023211883932418048
this is the vehicle index 16
1023160470560833536
this is the vehicle index 17
1027849379412705280
this is the vehicle index 18
1038357273237258240
this is the vehicle index 19
1038357867746295808
this i

In [12]:
df_all_trip_dtcs.head()

,Trip_ID,Start_Time,End_Time,DTC,status,time_of_dtc,description
0,1693369590000-1028649932849938432-100004183237...,1693369590000,1.693371e+12,810-14,active,2023-08-30T09:56:51.000Z,Speed Signal Input
1,1693337776000-1028649932849938432-100004183237...,1693337776000,1.693342e+12,P2454,active,2023-08-30T01:50:49.000Z,Particulate Filter Pressure sensor output belo...
2,1693337776000-1028649932849938432-100004183237...,1693337776000,1.693342e+12,P2454,active,2023-08-30T01:50:49.000Z,Particulate Filter Pressure sensor output belo...
3,1693369590000-1028649932849938432-100004183237...,1693369590000,1.693371e+12,810-14,active,2023-08-30T09:56:51.000Z,Speed Signal Input
4,1693337776000-1028649932849938432-100004183237...,1693337776000,1.693342e+12,P2454,active,2023-08-30T01:50:49.000Z,Particulate Filter Pressure sensor output belo...


In [17]:
# getting the count of common trips with both dtcs and alerts
vehicle_alerts = df_all_trip_alerts.Trip_ID.unique().tolist()
vehicle_dtcs = df_all_trip_dtcs.Trip_ID.unique().tolist()
common_trips = list(set(vehicle_alerts).intersection(vehicle_dtcs))
print(len(common_trips))


14


In [18]:
# joining the alert and dtc trips sharing the same Trip_ID
df_all_trip_alerts_dtcs = pd.merge(df_all_trip_alerts, df_all_trip_dtcs, on = ['Trip_ID', 'Start_Time', 'End_Time'], how= 'outer')


In [19]:
df_all_trip_alerts_dtcs.head()

,Trip_ID,Start_Time,End_Time,Predictive_alerts,severity,time_of alert,model,manufacturer,max_load_capacity,DTC,status,time_of_dtc,description
0,1693460631000-951132064558612480-9180628703342...,1693460631000,1.693462e+12,Engine operating temperature high,Minor,2023-08-31T11:15:49.000Z,furio 16T cargo BS6,mahindra,16000.0,NaN,NaN,NaN,NaN
1,1693360388000-951132064558612480-9180628703342...,1693360388000,1.693366e+12,Engine operating temperature high,Major,2023-08-30T08:22:42.000Z,furio 16T cargo BS6,mahindra,16000.0,NaN,NaN,NaN,NaN
2,1693456859000-1028649932849938432-100004183237...,1693456859000,1.693470e+12,Engine operating temperature high,Minor,2023-08-31T11:51:51.000Z,furio 16T cargo BS6,mahindra,16000.0,NaN,NaN,NaN,NaN
3,1697311906000-1028649932849938432-100004183237...,1697311906000,1.697316e+12,Engine operating temperature high,Major,2023-10-15T01:58:23.000Z,furio 16T cargo BS6,mahindra,16000.0,NaN,NaN,NaN,NaN
4,1697128633000-1028649932849938432-100004183237...,1697128633000,1.697137e+12,Engine operating temperature high,Major,2023-10-12T23:38:40.000Z,furio 16T cargo BS6,mahindra,16000.0,P2183,active,2023-10-13T00:04:09.000Z,Measured value of Coolant Temperature above limit


In [20]:
df_all_trip_alerts_dtcs.to_csv(os.path.join(output_path, "Trip_dtc_alerts.csv"))